## Deliverable 2. Create a Customer Travel Destinations Map.

In [61]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [62]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Atuona,PF,-9.8000,-139.0333,77.59,78,11,12.55,light rain
1,1,Sao Filipe,CV,14.8961,-24.4956,77.32,46,70,20.80,broken clouds
2,2,Muroto,JP,33.2833,134.1500,44.64,76,100,13.96,overcast clouds
3,3,Chuy,UY,-33.6971,-53.4616,68.36,95,98,8.48,overcast clouds
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,74.77,85,43,7.23,scattered clouds


In [63]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 25
What is the maximum temperature you would like for your trip? 60


In [64]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
2,2,Muroto,JP,33.2833,134.1500,44.64,76,100,13.96,overcast clouds
5,5,Aswan,EG,24.0934,32.9070,56.34,51,0,9.35,clear sky
6,6,Albany,US,42.6001,-73.9662,36.48,66,48,3.00,scattered clouds
13,13,Hanzhong,CN,33.0728,107.0303,36.27,75,8,1.83,clear sky
16,16,Sabha,LY,27.0377,14.4283,52.02,31,0,7.58,clear sky
18,18,Mataura,NZ,-46.1927,168.8643,51.48,89,98,5.03,overcast clouds
19,19,Dunedin,NZ,-45.8742,170.5036,50.81,86,100,11.99,light rain
24,24,Shahr-E Kord,IR,32.3256,50.8644,35.37,87,75,4.61,broken clouds
27,27,Kutum,SD,14.2000,24.6667,58.53,18,21,6.80,few clouds
34,34,Ushuaia,AR,-54.8000,-68.3000,47.86,53,20,24.16,few clouds


In [65]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        181
City           181
Country        181
Lat            181
Lng            181
Max Temp       181
Humidity       181
Cloudiness     181
Wind Speed     181
Description    181
dtype: int64

In [66]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
clean_df = preferred_cities_df

In [67]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Muroto,JP,44.64,overcast clouds,33.2833,134.1500,
5,Aswan,EG,56.34,clear sky,24.0934,32.9070,
6,Albany,US,36.48,scattered clouds,42.6001,-73.9662,
13,Hanzhong,CN,36.27,clear sky,33.0728,107.0303,
16,Sabha,LY,52.02,clear sky,27.0377,14.4283,
18,Mataura,NZ,51.48,overcast clouds,-46.1927,168.8643,
19,Dunedin,NZ,50.81,light rain,-45.8742,170.5036,
24,Shahr-E Kord,IR,35.37,broken clouds,32.3256,50.8644,
27,Kutum,SD,58.53,few clouds,14.2000,24.6667,
34,Ushuaia,AR,47.86,few clouds,-54.8000,-68.3000,


In [68]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": ""
}


# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [69]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [72]:
# 8a. Create the output File (CSV)
output_data_file = "weather_py_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [71]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [60]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))